# Data Wrangling
(this is unlikely to run unless you configure your system with a `DATA_DIR` environmental variable)

My goal here is to transform the masked data array into something that simulates a higher dimensional thing going on.

In [5]:
import os
import pandas as pd
import netCDF4 as nc
import plotly.express as px
import seaborn as sns
import numpy as np

Set up data directory from environmental variable.

In [2]:
data_dir = os.environ['DATA_DIR']
marks_fn = f'{data_dir}/remy_35_02_marks.nc'
position_fn = f'{data_dir}/remy_35_02_position_info.csv'

In [3]:
ds = nc.Dataset(marks_fn)
df = pd.read_csv(position_fn)

/nix/store/9hldfh2f48fiql7b5bkqm6z923lbiqyy-python3-3.7.6-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (11,12,13,15) have mixed types.Specify dtype option on import or set low_memory=False.



The goal for the end format is something like:

an array of floats in the shape of `(timestamps, n_channels)`
an array of ints in the shape of `(n_spikes,)`
an array of floats in the shape of `(timestamps,)`

The idea is we store the channels and positions of all the timestamps and we should be able to index them as a slice using the timestamp of each spike where a spike is defined as any time a spike is detected on one tetrode.

In [13]:
neural = ds['__xarray_dataarray_variable__']
neural.shape

(615851, 4, 24)

Create an array containing timestamp indices of spikes.
* convert nc format to numpy array with nans
* combine all tetrodes together by reshaping
* for each timestamp, set True if it contains a spike
* get the index of each timestamp containing a spike

None of this is reusable on different pieces of data yet. `615851` is how many timestamps. `4` is the number of channels on a tetrode. `24` is the number of tetrodes. `502559` is the number of spike events.

In [65]:
na = np.array(neural)
nar = na.reshape((615851, 4*24))
spikes_bool = np.logical_not(np.isnan(nar)).any(axis=1)
spikes = np.argwhere(spikes_bool).reshape((502559,)).shape

Wow about 80% of the time there's a spike going on.

In [67]:
502559/615851

0.8160399187465799

It has become apparent to me that the actual piece of data is not there. There is no underlying array that contains all of the data. I'll have to ask Eric for that.

In [93]:
unmasked = ds.variables['__xarray_dataarray_variable__'][:].data.reshape((615851, 4*24))

RuntimeError: NetCDF: HDF error

In [ ]:
positions